In [1]:
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict

In [2]:
torch. __version__

'1.13.0'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cpu')

In [5]:
df = pd.read_csv("../../data/her_molecules.csv")
print(f"size: {df.shape[0]}")
df.tail()

size: 2593


,name,smiles,IC50,units,activity,pIC50
2588,CHEMBL203661,O=C(C#CCCN1CCOCC1)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncn...,0.8,nM,high,8.903090
2589,CHEMBL203599,CN1CCN(CCC#CC(=O)Nc2cc3c(Nc4ccc(F)c(Cl)c4)ncnc...,0.7,nM,high,8.845098
2590,CHEMBL204638,O=C(C#CCCN1CCOCC1)Nc1cc2c(Nc3ccc(F)c(Br)c3)ncn...,0.7,nM,high,8.845098
2591,CHEMBL3344216,CN(C)[C@H](CS(C)(=O)=O)c1ccc(-c2ccc3ncnc(Nc4cc...,0.5,nM,high,8.698970
2592,CHEMBL437890,CN1CCN(CCC#CC(=O)Nc2cc3c(Nc4ccc(F)c(Br)c4)ncnc...,0.5,nM,high,8.698970


In [6]:
df.activity.unique()

array(['low', 'medium', 'high'], dtype=object)

In [7]:
df.activity.value_counts()

low       1097
high      1048
medium     448
Name: activity, dtype: int64

In [11]:
class CustomDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

In [12]:
if __name__ == "__main__":
    dataset = CustomDataset(df.smiles, df.activity)

In [13]:
from torch.utils.data import random_split

[0.8 *len(dataset)], [0.1 *len(dataset)], [0.1 *len(dataset)]

([2074.4], [259.3], [259.3])

In [14]:
train_dataset, test_dataset, val_dataset = random_split(dataset, [2075, 259, 259]) # 80, 10, 10
len(train_dataset + test_dataset + val_dataset)

2593

In [15]:
for smiles, labels in train_dataset:
    print("Input ID:\n " ,smiles)
    print("Label:\n" ,labels)
    break

Input ID:
  COc1cc2c(Nc3ccc(Cl)c(Cl)c3)ncnc2cc1OC[C@@H]1CN2CCC[C@@H]2CO1
Label:
 high


In [13]:
## https://github.com/topazape/LSTM_Chem/blob/master/lstm_chem/utils/smiles_tokenizer2.py

class SmilesTokenizer(object):
    def __init__(self):
        atoms = [
            'Al', 'As', 'B', 'Br', 'C', 'Cl', 'F', 'H', 'I', 'K', 'Li', 'N',
            'Na', 'O', 'P', 'S', 'Se', 'Si', 'Te'
        ]
        special = [
            '(', ')', '[', ']', '=', '#', '%', '0', '1', '2', '3', '4', '5',
            '6', '7', '8', '9', '+', '-', 'se', 'te', 'c', 'n', 'o', 's'
        ]

        self.table = sorted(atoms, key=len, reverse=True) + special 

        self.table_2_chars = list(filter(lambda x: len(x) == 2, self.table))
        self.table_1_chars = list(filter(lambda x: len(x) == 1, self.table))
        self.vocab_dict = {}

    def tokenize(self, smiles):
        smiles = smiles + ' '
        N = len(smiles)
        token = []
        i = 0
        while (i < N):
            c1 = smiles[i]
            c2 = smiles[i:i + 2]

            if c2 in self.table_2_chars:
                token.append(c2)
                i += 2
                continue

            if c1 in self.table_1_chars:
                token.append(c1)
                i += 1
                continue

            i += 1

        return np.asarray(token, dtype=object)
        
    def vocaburaly(self):
        vocab_dict = {}
        for i, tok in enumerate(self.table):
            vocab_dict[tok] = i
        return vocab_dict
    
    def index_encode(self, tokenized_smiles):
        vocab_dict = {}
        for i, tok in enumerate(self.table):
            vocab_dict[tok] = i
        encoded = [vocab_dict[t] for t in tokenized_smiles ]
        return encoded

In [14]:
tokenizer = SmilesTokenizer()
tokens = [tokenizer.tokenize(x) for x in df.smiles]
vocabulary = tokenizer.vocaburaly()
indexed_smiles = [tokenizer.index_encode(x) for x in tokens]

In [23]:
vocabulary

{'Al': 0,
 'As': 1,
 'Br': 2,
 'Cl': 3,
 'Li': 4,
 'Na': 5,
 'Se': 6,
 'Si': 7,
 'Te': 8,
 'B': 9,
 'C': 10,
 'F': 11,
 'H': 12,
 'I': 13,
 'K': 14,
 'N': 15,
 'O': 16,
 'P': 17,
 'S': 18,
 '(': 19,
 ')': 20,
 '[': 21,
 ']': 22,
 '=': 23,
 '#': 24,
 '%': 25,
 '0': 26,
 '1': 27,
 '2': 28,
 '3': 29,
 '4': 30,
 '5': 31,
 '6': 32,
 '7': 33,
 '8': 34,
 '9': 35,
 '+': 36,
 '-': 37,
 'se': 38,
 'te': 39,
 'c': 40,
 'n': 41,
 'o': 42,
 's': 43}

In [19]:
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab1 = vocab(ordered_dict)
vocab1.insert_token("<pad>", 0)
vocab1.insert_token("<unk>", 1)
vocab1.set_default_index(1)

In [21]:
ordered_dict

OrderedDict([('s', 43),
             ('o', 42),
             ('n', 41),
             ('c', 40),
             ('te', 39),
             ('se', 38),
             ('-', 37),
             ('+', 36),
             ('9', 35),
             ('8', 34),
             ('7', 33),
             ('6', 32),
             ('5', 31),
             ('4', 30),
             ('3', 29),
             ('2', 28),
             ('1', 27),
             ('0', 26),
             ('%', 25),
             ('#', 24),
             ('=', 23),
             (']', 22),
             ('[', 21),
             (')', 20),
             ('(', 19),
             ('S', 18),
             ('P', 17),
             ('O', 16),
             ('N', 15),
             ('K', 14),
             ('I', 13),
             ('H', 12),
             ('F', 11),
             ('C', 10),
             ('B', 9),
             ('Te', 8),
             ('Si', 7),
             ('Se', 6),
             ('Na', 5),
             ('Li', 4),
             ('Cl', 3),
             ('

In [17]:
sorted_by_freq_tuples[:7]

[('s', 43), ('o', 42), ('n', 41), ('c', 40), ('te', 39), ('se', 38), ('-', 37)]

In [18]:
vocab1.get_itos()[:10]

['<pad>', '<unk>', 's', 'o', 'n', 'c', 'te', 'se', '-', '+']

In [19]:
print(f"Vocabulary size: {len(vocab1)}")

Vocabulary size: 45


In [20]:
num_class = len(set([label for (text, label) in train_dataset]))
print(f"Number of classes: {num_class}")

Number of classes: 3


In [21]:
text_pipeline = lambda x: [vocab1[token] for token in tokenizer.tokenize(x)]
text_pipeline("O=S(=O)")

[29, 22, 27, 26, 22, 29, 25]

In [22]:
label_pipeline = lambda x: 0 if x == "low" else (1 if x == "medium" else 2)
label_pipeline("low")

0

In [23]:
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for   _text, _label in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text),
                                      dtype=torch.long)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list,
                                 dtype=torch.long)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(text_list,
                                                     batch_first=True)  
    return padded_text_list, label_list, lengths

In [24]:
collate_batch(val_dataset)

(tensor([[35, 35, 26,  ...,  0,  0,  0],
         [35, 29,  5,  ...,  0,  0,  0],
         [30, 35, 26,  ...,  0,  0,  0],
         ...,
         [29, 22, 35,  ...,  0,  0,  0],
         [35, 27, 26,  ...,  0,  0,  0],
         [29, 22, 35,  ...,  0,  0,  0]]),
 tensor([2, 2, 2, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 0, 2, 1, 1, 0, 0, 2, 2, 0, 1, 2,
         2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 0, 2, 0, 0, 0, 1, 0, 2, 2, 0,
         0, 2, 0, 2, 2, 0, 2, 2, 2, 1, 2, 2, 0, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0,
         1, 0, 2, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0,
         2, 0, 1, 2, 0, 1, 0, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0,
         2, 2, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2,
         1, 2, 0, 0, 2, 0, 2, 0, 1, 2, 2, 1, 2, 2, 0, 2, 0, 0, 1, 1, 0, 2, 2, 2,
         2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 1, 0, 0, 2, 2,
         2, 2, 0, 1, 0, 2, 0, 0, 2, 0, 2, 0, 0, 1, 2, 2, 0, 1, 0, 0, 2, 1, 2, 2,
         

In [25]:
train_dataloader = DataLoader(train_dataset,batch_size=10, shuffle=True, collate_fn=collate_batch)
val_dataloader = DataLoader(val_dataset,batch_size=10, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True, collate_fn=collate_batch)

In [26]:
text_batch, label_batch, length_batch = next(iter(test_dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[35, 29,  5, 18,  5,  5, 17,  5, 26, 30,  5, 16,  5,  5,  5, 26,  8,  5,
         15,  4,  5, 14,  5,  5,  5,  5,  5, 14,  2, 15, 25,  5,  5, 16, 25,  4,
          5,  4,  5, 17,  5,  5, 18, 29, 35, 35, 35, 30, 18, 35, 35, 30, 26, 35,
         25, 35, 35, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [35, 29,  5, 18,  5,  5, 26, 30, 17, 35, 35, 35, 26, 30, 16, 35, 35, 35,
         35, 35, 16, 25, 35, 35, 17, 25,  5, 26, 42, 25,  5,  5, 18, 30,  5, 18,
          4,  5,  5,  5, 26,  8,  5, 17,  5, 26,  8,  5, 16,  5,  5,  5, 26, 29,
         35, 25,  5, 26, 35, 26, 22, 29, 25, 30,  5, 15,  5, 26, 34, 25,  5,  5,
          5,  5, 15, 34, 25,  5, 16, 25,  4,  5, 16,  5,  5,  5,  5,  4, 17, 16,
         25,  4, 18],
        [35, 35, 35, 35, 30, 26, 35,  5, 18,  5,  5, 26, 43, 25,  5,  5, 26, 43,
         25,  5, 18, 29, 25, 35, 26, 22, 27, 25, 30,  5, 18,  5, 

In [27]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size,
                 num_layers, fc_hidden_size, output_size):
        super(). __init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, input_dim (embedding))
        # batch_dim = number of samples per batch
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, num_layers,
                           batch_first=True, bidirectional=True) 
        # only use the hidden_size, since its a many to one
        self.fc1 = nn.Linear(rnn_hidden_size * num_layers, fc_hidden_size)  
        
        self.relu = nn.ReLU()
        
        self.fc2 = nn.Linear(fc_hidden_size, output_size) 
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        # text dim: [sentence length, batch size]
        # text_length = [batch size]
        
        # [sentence len, batch size] => [sentence len, batch size, embedding size]
        out = self.embedding(text)
        
        # pack sequence to avoid using <paddings> during computations (saves computations)
        # lengths needs to be on cpu
        out = nn.utils.rnn.pack_padded_sequence(out, text_lengths.to('cpu'),
                                                enforce_sorted=False, batch_first=True)
        # =>  [batch_size, sentence len,  embedding dim]
        
        # Propagate input through LSTM
        out, (hidden, cell) = self.rnn(out) ## lstm with input, hidden, and internal (cell) state
        # out dim: [batch size, sentence length, hidden dim]
        # cell dim: [num layers, batch size, hidden dim]
        # hidden dim: [num_layers, batch_size, hidden dim]
        
        # final layer foward hidden state     
        hidden_fwd = hidden[-2]
        # final layer backwaed hidden state 
        hidden_bck = hidden[-1]
        
        # concatenate the 2 layers to pass to linear layer
        # hidden_fwd/bck = [batch size, hid dim]
        out = torch.cat((hidden_fwd, hidden_bck), dim = 1)
        # out = [batch size, hid dim * 2]

        out = self.fc1(out) ## first dense layer
        out = self.relu(out)
        out = self.fc2(out) ## final layer
        out = self.sigmoid(out) 
        return out

Here, I'll instantiate the network. First up, defining the hyperparameters.

    vocab_size: Size of our vocabulary or the range of values for our input, word tokens.
    output_size: Size of our desired output; the number of class scores we want to output.
    embedding_dim: Number of columns in the embedding lookup table; size of our embeddings.
    num_filters: Number of filters that each convolutional layer produces as output.
    filter_sizes: A list of kernel sizes; one convolutional layer will be created for each kernel size.


In [28]:
vocab_size = len(vocab1) ## len of vocab size
embed_dim = 70 ## input size, usually around 50-500 dimensions
num_layers = 2 ## number of recurrent layers, 2 would mean stacking 2 layers to form stacked LSTM
rnn_hidden_size = 100 ## usually around 100-500 dimensions
fc_hidden_size = 100 ## usually around 100-500 dimensions
output_size = len(df.activity.unique()) ## num_classes 3

In [29]:
## Now we will instantiate the class LSTM1 object.

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size,
            num_layers, fc_hidden_size, output_size)

In [30]:
model

RNN(
  (embedding): Embedding(45, 70, padding_idx=0)
  (rnn): LSTM(70, 100, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=200, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=3, bias=True)
  (sigmoid): Sigmoid()
)

In [31]:
def train(dataloader):
    # model training mode (gradient computation)
    model.train()
    # initailiz acc, and loss at zero 
    total_acc, total_loss = 0, 0
    for idx, (text, label, length) in enumerate (dataloader):
        # reset gradients to zero before each instance
        optimizer.zero_grad()
        # label predictions (forward papagation)
        # squeeze(1) => drop superficial one dimensional from a tensor
        predicted_label = model(text, length).squeeze(1) # or [:,0]
        # loss calculation
        loss = loss_fn(predicted_label, label)
        # compute gradients (backward propagation) 
        # to minimize loss functions with gradient descent
        loss.backward()
        # update parameters based on the computed gradients
        optimizer.step()
        # logging
        if not idx % 50:
            print(f"Epoch: {epoch + 1:04d}/{num_epochs:0d} | "
                  f"Batch {idx:03d}/{len(dataloader):03d} | "
                  f"Loss: {loss:.4f}")
        # compute total accuracy
        # return an indice of the max value of all elements
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        # compute total loss after back prop and parameter update
        total_loss += loss.item() * label.size(0)
    # compare true labels with the predicted labels to compute accuracy
    return total_acc/len(dataloader.dataset), \
            total_loss/len(dataloader.dataset)

In [32]:
def evaluate(dataloader):
    # model evaluation mode (no gradient computation)
    model.eval()
    # initailize acc, and loss at zero 
    total_acc, total_loss = 0, 0
    # disabling gradient calculation
    with torch.no_grad():
        for text, label, length in dataloader:
            # label predictions (forward papagation)
            # squeeze(1) => drop superficial one dimensional from a tensor
            predicted_label = model(text, length).squeeze(1) # reshape
            # loss calculation
            loss = loss_fn(predicted_label, label)
            # compute total accuracy
            # return an indice of the max value of all elements
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            # compute total loss after back prop and parameter update
            total_loss += loss.item() * label.size(0)
        # compare true labels with the predicted labels to compute accuracy
        return total_acc/len(dataloader.dataset), \
                total_loss/len(dataloader.dataset)

In [33]:
# for multiclass classification we use
loss_fn = nn.CrossEntropyLoss()
# Adam Optimizer to update parameters based on the computed gradients
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [34]:
model = model.to(device)
loss_fn = loss_fn.to(device)

In [35]:
torch.manual_seed(1)
num_epochs = 10

start_time = time.time()
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dataloader)
    acc_val , loss_val = evaluate(val_dataloader)
    print(f"Train Acc.: {100 * acc_train:.2f}%"
          f"\nValid Acc.: {100 * acc_val:.2f}%")
    print(f'Time elapsed: {(time.time() - start_time) / 60:.2f} min')

Epoch: 0001/10 | Batch 000/208 | Loss: 1.1002
Epoch: 0001/10 | Batch 050/208 | Loss: 1.0924
Epoch: 0001/10 | Batch 100/208 | Loss: 1.0496
Epoch: 0001/10 | Batch 150/208 | Loss: 0.9923
Epoch: 0001/10 | Batch 200/208 | Loss: 1.0882
Train Acc.: 51.23%
Valid Acc.: 59.85%
Time elapsed: 0.60 min
Epoch: 0002/10 | Batch 000/208 | Loss: 0.9923
Epoch: 0002/10 | Batch 050/208 | Loss: 1.0715
Epoch: 0002/10 | Batch 100/208 | Loss: 1.0009
Epoch: 0002/10 | Batch 150/208 | Loss: 0.8059
Epoch: 0002/10 | Batch 200/208 | Loss: 1.0082
Train Acc.: 58.36%
Valid Acc.: 60.62%
Time elapsed: 1.19 min
Epoch: 0003/10 | Batch 000/208 | Loss: 1.1499
Epoch: 0003/10 | Batch 050/208 | Loss: 0.8551
Epoch: 0003/10 | Batch 100/208 | Loss: 0.8107
Epoch: 0003/10 | Batch 150/208 | Loss: 0.6906
Epoch: 0003/10 | Batch 200/208 | Loss: 0.8300
Train Acc.: 60.72%
Valid Acc.: 62.93%
Time elapsed: 1.80 min
Epoch: 0004/10 | Batch 000/208 | Loss: 0.8862
Epoch: 0004/10 | Batch 050/208 | Loss: 1.1139
Epoch: 0004/10 | Batch 100/208 | Lo

In [36]:
acc_test, _ = evaluate(test_dataloader)
print(f"Test Acc.: {100 * acc_test:.2f}%")

Test Acc.: 68.73%


In [ ]:
# The only difference here is that instead of using a sigmoid function to squash the input between 0 and 1,
# we use the argmax to get the highest predicted class index. 
# or we use sofmax activation function and squash the input to range 0 and 1 and sum them to 1
# in this case we get both the class label pred and the predicted probability

In [37]:
## https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
## https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2_lstm.ipynb

sentiment_label = {0: "low",
                   1: "medium",
                   2: "high"}

def predict(text, text_pipeline):
    # evaluation mode (no gradients)
    with torch.no_grad():
        # tokenize and index the tokens
        processed_text = torch.tensor(text_pipeline(text))
        # add a batch dimension
        processed_text = processed_text.unsqueeze(0).to(device)
        # compute sequence length
        text_length = processed_text.size(0)
        # convert to tensor and add a batch dimension
        text_length = torch.tensor(text_length).unsqueeze(0)
        # prediction
        prediction = model(processed_text, text_length)
        # reduction real numbers to values between 0 and 1
        probability = torch.softmax(prediction, dim=1)
        # get the max value of all elements
        predicted_probability, predicted_class, = torch.max(probability, dim=1)
        # convert tensor holding a single value into an integer
        return predicted_class.item(), predicted_probability.item()

In [38]:
text = "O=C(C#CCCN1CCOCC1)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncn"
pred_class, pred_proba = predict(text, text_pipeline)

print(f'Predicted Class: {pred_class} = {sentiment_label[pred_class]}')
print(f'Probability: {pred_proba}')

Predicted Class: 2 = high
Probability: 0.5652756690979004


In [39]:
text = "Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)[C@@H](O)[C@H]1O"
pred_class, pred_proba = predict(text, text_pipeline)

print(f'Predicted Class: {pred_class} = {sentiment_label[pred_class]}')
print(f'Probability: {pred_proba}')

Predicted Class: 0 = low
Probability: 0.4145854711532593
